### Merge Multimodal Embeddings from Different Media

In [7]:
import pandas as pd
import os
from json import loads, dumps
import tempfile, shutil

os.chdir(os.getcwd())
file_path_1 = 'FinalEmbeddingResults/videoembeddings_file12082024.json'
file_path_2 ='FinalEmbeddingResults/summarycontent_image_embeddings_v1.json'

df_video = pd.read_json(file_path_1, lines=True)
df_video=df_video.rename(columns={"video path": "path", 
                                  "video_embedding": "multimodal_embedding"
                       })
df_video['media_type']=['video']*len(df_video)

df_image = pd.read_json(file_path_2, lines=True)[['id',	"image path",'image embedding']]
df_image=df_image.rename(columns={"image path": "path",                                
                                  "image embedding": "multimodal_embedding",
                       })
df_image['media_type']=['image']*len(df_image)
df_multimodal = pd.concat([df_video,df_image],axis=0 )

df_multimodal.head()

,id,path,multimodal_embedding,start_offset_sec_embedding,end_offset_sec_embedding,media_type
0,60MI24_1_A_HBB.mp4,gs://raw_nine_files/60MI24_1_A_HBB.mp4,"[-0.016659908, 0.0433404, -0.000906404282, -0....",0.0,16.0,video
1,60MI24_1_A_HBB.mp4,gs://raw_nine_files/60MI24_1_A_HBB.mp4,"[-0.0394997261, 0.0360285975, -0.00222768984, ...",16.0,32.0,video
2,60MI24_1_A_HBB.mp4,gs://raw_nine_files/60MI24_1_A_HBB.mp4,"[-0.033938162, 0.044598572, -0.0061921021, -0....",32.0,48.0,video
3,60MI24_1_A_HBB.mp4,gs://raw_nine_files/60MI24_1_A_HBB.mp4,"[-0.0169373192, 0.0542577393, -0.00920145586, ...",48.0,64.0,video
4,60MI24_1_A_HBB.mp4,gs://raw_nine_files/60MI24_1_A_HBB.mp4,"[-0.010487088, 0.0627433956, -0.00739834085, -...",64.0,80.0,video


In [8]:
result = df_multimodal.to_json(orient="records")
parsed = loads(result)
json_content=dumps(parsed, indent=4)  


# Create temporary file to write summaries to
multimodal_embeddings_file = tempfile.NamedTemporaryFile(suffix=".json", delete=False)
with  open(multimodal_embeddings_file.name, "a") as sf:     
  sf.writelines(json_content) 

multimodal_embeddings_file_name = multimodal_embeddings_file.name
multimodal_embeddings_file.close()
shutil.copy(multimodal_embeddings_file_name, 'multimodal_embeddings.json')


'multimodal_embeddings.json'

### create bucket and load data into bucket

In [9]:
#set project info
PROJECT = !gcloud config get-value project
PROJECT_ID = PROJECT[0]
REGION = "us-central1"

In [40]:
#create bucket if not exists

from google.cloud import storage
from google.cloud.storage import Bucket
bucket_name=f"artifacts-{PROJECT_ID}-embeddings"
BUCKET_URI = f"gs://{bucket_name}"  # @param {type:"string"}
client = storage.Client()
exists = Bucket(client, bucket_name).exists()

if not exists:
    print('creating bucket..')
    #set bucket info to create a bucket
    ! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

In [38]:
#we have to remove [] from json file to be able to load it into bigquery

!cat multimodal_embeddings.json | jq -c '.[]' > multimodal_embeddings_V1.json

In [41]:
#embeddings
UNIQUE_FOLDER_NAME='multimodal_embeddings'
EMBEDDINGS_INITIAL_URI = f"{BUCKET_URI}/{UNIQUE_FOLDER_NAME}/"
! gsutil cp multimodal_embeddings_V1.json {EMBEDDINGS_INITIAL_URI}

Copying file://multimodal_embeddings_V1.json [Content-Type=application/json]...
/ [1 files][  4.3 MiB/  4.3 MiB]                                                
Operation completed over 1 objects/4.3 MiB.                                      
